In [ ]:
import requests
import pandas as pd

path = input("Podaj ścieżkę do pliku: ")

exclude = []

with open(path) as f:

  for line in f:
    exclude.append(line.strip())


print(exclude)
exclude_bioprojects = [x for x in exclude if x.startswith("PRJ")]

query = "(gut microbiome[mesh] AND stool[mesh] AND shotgun[mesh])"
for prj in exclude:
    query += f" NOT {prj}[BioProject]"

base_esearch = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"
params_esearch = {
    "db": "bioproject",
    "term": query,
    "retmode": "json",
    "retmax": 10000,
    "usehistory": "y",
    "tool": "microbiome_pipeline",
    "email": "ola.pacult@gmail.com"
}

r = requests.get(base_esearch, params=params_esearch).json()
query_key = r["esearchresult"]["querykey"]
webenv = r["esearchresult"]["webenv"]


base_summary = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi"
params_summary = {
    "db": "bioproject",
    "query_key": query_key,
    "WebEnv": webenv,
    "retmode": "json",
    "retmax": 10000
}

r = requests.get(base_summary, params=params_summary).json()
records = r['result']
uids = records['uids']

bio_projects_accessions = []

for uid in uids:
  bio_projects_accessions.append(records[uid]['project_acc'])

len(bio_projects_accessions)





Podaj ścieżkę do pliku: all_study_accessions.txt
['PRJEB27928', 'PRJNA290729', 'PRJNA397112', 'PRJNA278393', 'PRJEB33013', 'PRJNA638405', 'PRJNA890008', 'PRJNA449784', 'PRJDB3601', 'PRJNA602729', 'PRJNA598446', 'PRJEB29127', 'PRJEB1690', 'PRJEB28543', 'PRJEB4336', 'PRJNA389280', 'PRJNA398089', 'PRJNA504891', 'PRJEB19090', 'PRJEB12947', 'PRJEB6337', 'PRJNA422434', 'PRJEB1220', 'PRJEB24557', 'PRJNA400072', 'PRJNA354235', 'PRJNA763023', 'PRJNA268964', 'PRJNA384246', 'PRJNA48479', 'PRJNA638404', 'PRJNA447983', 'PRJEB6070', 'PRJNA834801', 'PRJNA373901', 'PRJEB6456', 'PRJNA638403', 'PRJNA385949', 'PRJNA429097', 'PRJEB27005', 'PRJEB8094', 'PRJNA373879', 'PRJEB10878', 'PRJEB17784', 'PRJEB12124', 'PRJNA690543', 'PRJNA602731', 'PRJNA299502', 'PRJEB11532', 'PRJNA529124', 'PRJNA665061', 'PRJNA328899', 'PRJNA475246', 'PRJEB9576', 'PRJNA392180', 'PRJNA688274', 'PRJNA340216', 'PRJNA421881', 'PRJNA832909', 'PRJEB39223', 'PRJNA395744', 'PRJNA375935', 'PRJNA319574', 'PRJEB12449', 'PRJEB1786', 'PRJEB2152

98

In [ ]:
pip install pysradb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.0/177.0 kB 3.6 MB/s eta 0:00:00


In [ ]:
%%bash
apt-get update && apt-get install -y sra-toolkit

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://cli.github.com/packages stable InRelease [3,917 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,412 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,275 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [5,712 kB]
Get:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:13 https://ppa.launchpadcontent.net/graphi

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [ ]:
from pysradb.sraweb import SRAweb
import subprocess
from pathlib import Path



/usr/local/lib/python3.12/dist-packages/pysradb/utils.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [ ]:
from pysradb.sraweb import SRAweb
import subprocess
from pathlib import Path


fastq_dir = Path("fastq")
fastq_dir.mkdir(exist_ok=True)


db = SRAweb()

all_runs = []
runs_no_found = 0
# test for smaller amount of data
test_bio_projects = bio_projects_accessions[:2]

for study in test_bio_projects:

    try:
        df = db.sra_metadata(study, detailed=True)

        runs = df["run_accession"].unique().tolist()
        all_runs.extend(runs)

    except Exception as e:
        runs_no_found+=1


print(f"found {len(all_runs)} run_accessions (SRR). Number of no found: {runs_no_found}")


all_runs = all_runs[:2]
for run in all_runs:
    print(f"downlading FASTQ for {run} ...")

    subprocess.run([
        "fasterq-dump",
        run,
        "-O", str(fastq_dir),
        "--split-files",
        "--threads", "4"
    ])


found 32 run_accessions (SRR). Number of no found: 1
downlading FASTQ for ERR15462220 ...
downlading FASTQ for ERR15462221 ...


"\n\n\n  Błąd przy pobieraniu dla PRJEB76988: 'NoneType' object is not subscriptable\nPobieranie run_accessions dla PRJEB76987 ...\nNo results found for PRJEB76987\n  Błąd przy pobieraniu dla PRJEB76987: 'NoneType' object is not subscriptable\nPobieranie run_accessions dla PRJEB76938 ...\nNo results found for PRJEB76938\n  Błąd przy pobieraniu dla PRJEB76938: 'NoneType' object is not subscriptable\nPobieranie run_accessions dla PRJNA1131664 ...\n  Znaleziono 15 SRR\nPobieranie run_accessions dla PRJNA1130109 ...\n  Znaleziono 203 SRR\nPobieranie run_accessions dla PRJEB72433 ...\n  Znalezio\n\n\n  "

In [ ]:
from pysradb.sraweb import SRAweb
import subprocess
from pathlib import Path


fastq_dir = Path("fastq")
fastq_dir.mkdir(exist_ok=True)


db = SRAweb()

all_runs = []
runs_no_found = 0
for study in bio_projects_accessions:

    try:
        df = db.sra_metadata(study, detailed=True)
        runs = df["run_accession"].unique().tolist()
        all_runs.extend(runs)
    except Exception as e:
        runs_no_found+=1

print(f"found {len(all_runs)} run_accessions (SRR).and no found{runs_no_found}")


for run in all_runs:
    print(f"downlading FASTQ for {run} ...")

    subprocess.run([
        "fasterq-dump",
        run,
        "-O", str(fastq_dir),
        "--split-files",
        "--threads", "4"
    ])
